In [ ]:
!rm -rf /root/.cache/torch/text/datasets/IMDB

In [ ]:
!pip uninstall torch torchtext -y
!pip install torch torchtext --index-url https://download.pytorch.org/whl/cu118
!pip install 'portalocker>=2.0.0'
!pip install 'numpy<2'

Found existing installation: torch 2.2.0+cu118
Uninstalling torch-2.2.0+cu118:
  Successfully uninstalled torch-2.2.0+cu118
Found existing installation: torchtext 0.17.0+cpu
Uninstalling torchtext-0.17.0+cpu:
  Successfully uninstalled torchtext-0.17.0+cpu
Looking in indexes: https://download.pytorch.org/whl/cu118
  Using cached https://download.pytorch.org/whl/cu118/torch-2.7.0%2Bcu118-cp311-cp311-manylinux_2_28_x86_64.whl.metadata (28 kB)
  Using cached https://download.pytorch.org/whl/torchtext-0.17.0%2Bcpu-cp311-cp311-linux_x86_64.whl (2.0 MB)
  Using cached https://download.pytorch.org/whl/sympy-1.13.3-py3-none-any.whl.metadata (12 kB)
  Using cached https://download.pytorch.org/whl/cu118/nvidia_cudnn_cu11-9.1.0.70-py3-none-manylinux2014_x86_64.whl (663.9 MB)
  Using cached https://download.pytorch.org/whl/cu118/nvidia_nccl_cu11-2.21.5-py3-none-manylinux2014_x86_64.whl (147.8 MB)
  Using cached https://download.pytorch.org/whl/triton-3.3.0-cp311-cp311-manylinux_2_27_x86_64.manylin

Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/cli/base_command.py", line 179, in exc_logging_wrapper
    status = run_func(*args)
             ^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/cli/req_command.py", line 67, in wrapper
    return func(self, options, args)
           ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/commands/install.py", line 362, in run
^C


In [ ]:
import sys
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

In [ ]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
import torch.nn as nn
import torch.optim as optim
from torchtext.datasets import IMDB
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence
import torch.nn.functional as F
import torch.quantization
import math # For positional encoding
from torch.cuda.amp import autocast, GradScaler # For mixed precision


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/usr/local/lib/python3.11/dist-packages/colab_kernel_launcher.py", line 37, in <module>
    ColabKernelApp.launch_instance()
  File "/usr/local/lib/python3.11/dist-packages/traitlets/config/application.py", line 992, in launch_instance
    app.start()
  File "/usr/local/lib/python3.11/dist-packages/ipykernel/kernelapp.py", line 712, in start
    self.io_loop.start()
  File "/usr/local/lib/python3.11/dist-package

Using device: cuda


In [ ]:
# --- 0. Setup Global Variables and Special Tokens ---
UNK_IDX, PAD_IDX, BOS_IDX, EOS_IDX = 0, 1, 2, 3
special_tokens = ['<unk>', '<pad>', '<bos>', '<eos>']

In [ ]:
# --- 1. Data Loading and Preprocessing ---
train_iter, test_iter = IMDB(split=('train', 'test'))
tokenizer = get_tokenizer('basic_english')

def yield_tokens(data_iter):
    for _, text in data_iter:
        yield tokenizer(text)

vocab = build_vocab_from_iterator(yield_tokens(train_iter),
                                  min_freq=1,
                                  specials=special_tokens,
                                  special_first=True)
vocab.set_default_index(UNK_IDX)

def text_pipeline(text):
    return vocab(tokenizer(text))

MAX_SEQUENCE_LENGTH = 128 # Reduced for memory efficiency

def collate_batch(batch):
    data = []
    for _, text in batch:
        token_ids = [BOS_IDX] + text_pipeline(text) + [EOS_IDX]
        if len(token_ids) > MAX_SEQUENCE_LENGTH:
            token_ids = token_ids[:MAX_SEQUENCE_LENGTH - 1] + [EOS_IDX]
        data.append(torch.tensor(token_ids, dtype=torch.long))

    data = pad_sequence(data, batch_first=True, padding_value=PAD_IDX)

    # For Transformer: input `src` (tokens up to t-1), target `tgt` (tokens from t)
    # Plus padding mask for `src`
    src = data[:, :-1]
    tgt = data[:, 1:]

    # src_padding_mask: True where PAD_IDX
    src_padding_mask = (src == PAD_IDX) # Boolean mask: (batch_size, seq_len)

    return src, tgt, src_padding_mask # Return input, target, and padding mask

In [ ]:
# --- Helper for Causal Mask (needed for autoregressive Transformer) ---
def generate_square_subsequent_mask(sz):
    """Generates an upper-triangular matrix of -inf, with 0s on diag & lower."""
    mask = (torch.triu(torch.ones(sz, sz)) == 1).transpose(0, 1)
    mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
    return mask

In [ ]:
# --- Positional Encoding (common in Transformers) ---
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, dropout=0.1, max_len=5000):
        super().__init__()
        self.dropout = nn.Dropout(p=dropout)
        position = torch.arange(max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2) * (-math.log(10000.0) / d_model))
        pe = torch.zeros(max_len, 1, d_model)
        pe[:, 0, 0::2] = torch.sin(position * div_term)
        pe[:, 0, 1::2] = torch.cos(position * div_term)
        self.register_buffer('pe', pe)

    def forward(self, x):
        """
        Args:
            x: Tensor, shape (batch_size, seq_len, embed_dim)
        """
        # Adjust PE shape to match batch_first (x.size(1) is seq_len)
        x = x + self.pe[:x.size(1)].transpose(0,1)
        return self.dropout(x)

In [ ]:
# --- 2. Model Definition (Transformer Text Generator) ---
class TransformerTextGenerator(nn.Module):
    def __init__(self, vocab_size, embed_dim, nhead, num_layers, dim_feedforward, dropout=0.5):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim, padding_idx=PAD_IDX)
        self.pos_encoder = PositionalEncoding(embed_dim, dropout)

        # Transformer Decoder Layer: building block
        decoder_layer = nn.TransformerDecoderLayer(d_model=embed_dim, nhead=nhead,
                                                 dim_feedforward=dim_feedforward, dropout=dropout,
                                                 batch_first=True)
        # Transformer Decoder: stacks multiple decoder layers
        self.transformer_decoder = nn.TransformerDecoder(decoder_layer, num_layers)

        self.fc = nn.Linear(embed_dim, vocab_size)
        self.init_weights()
        self.nhead = nhead # Store nhead for later use if needed (e.g. for generation hidden state handling)

    def init_weights(self):
        initrange = 0.1
        self.embedding.weight.data.uniform_(-initrange, initrange)
        self.fc.bias.data.zero_()
        self.fc.weight.data.uniform_(-initrange, initrange)

    def forward(self, src, src_mask, src_padding_mask):
        """
        Args:
            src: Tensor, shape (batch_size, seq_len)
            src_mask: Tensor, shape (seq_len, seq_len) for causal masking
            src_padding_mask: Tensor, shape (batch_size, seq_len) for padding
        """
        src = self.embedding(src) * math.sqrt(self.embedding.embedding_dim) # Scale embedding
        src = self.pos_encoder(src)

        # Transformer Decoder takes:
        # tgt: target sequence (here, same as src for language modeling)
        # memory: encoder output (not used directly here, but memory_mask needed)
        # tgt_mask: causal mask (prevents attending to future tokens)
        # memory_mask: mask for memory (not used here, but for consistency set same as tgt_mask)
        # tgt_key_padding_mask: mask for padding in tgt
        # memory_key_padding_mask: mask for padding in memory (not used here)
        output = self.transformer_decoder(tgt=src, memory=src, # For decoder-only LM, tgt and memory are the same
                                          tgt_mask=src_mask,
                                          memory_mask=src_mask,
                                          tgt_key_padding_mask=src_padding_mask,
                                          memory_key_padding_mask=src_padding_mask)

        output = self.fc(output) # Map to vocabulary logits
        return output

# Model parameters
VOCAB_SIZE = len(vocab)
EMBED_DIM = 64
NHEAD = 8 # Number of attention heads
NUM_LAYERS = 3 # Number of transformer decoder layers
DIM_FEEDFORWARD = 128 # Hidden dimension of the feedforward network in each transformer layer
DROPOUT = 0.1

model = TransformerTextGenerator(VOCAB_SIZE, EMBED_DIM, NHEAD, NUM_LAYERS, DIM_FEEDFORWARD, DROPOUT).to(device)

In [ ]:
# --- 3. Training Setup ---
BATCH_SIZE = 64 # Keep low due to memory
train_dataloader = DataLoader(train_iter, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_batch)

print("Starting training...")

criterion = nn.CrossEntropyLoss(ignore_index=PAD_IDX)
optimizer = optim.Adam(model.parameters(), lr=0.0005) # Slightly lower LR often helpful for Transformers
num_epochs = 2 # Increase for better performance

# For Mixed Precision Training
scaler = GradScaler() # Initialize scaler once

Starting training...


In [ ]:
# --- 4. Training Loop ---
def train(dataloader, model, criterion, optimizer, epoch):
    model.train()
    total_loss = 0
    total_batches = 0
    for idx, (data, targets, src_padding_mask) in enumerate(dataloader): # Unpack new return values
        data, targets, src_padding_mask = data.to(device), targets.to(device), src_padding_mask.to(device)

        # Create causal mask for the current sequence length
        src_mask = generate_square_subsequent_mask(data.size(1)).to(device)

        optimizer.zero_grad()
        with autocast():
            # Pass all masks to the model
            predicted_logits = model(data, src_mask, src_padding_mask)
            # Reshape logits for CrossEntropyLoss: (N, C) and (N)
            loss = criterion(predicted_logits.reshape(-1, VOCAB_SIZE), targets.reshape(-1))

        scaler.scale(loss).backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.5) # Clip gradients to prevent exploding gradients
        scaler.step(optimizer)
        scaler.update()

        del data, targets, predicted_logits, src_mask, src_padding_mask
        torch.cuda.empty_cache()

        total_loss += loss.item()
        total_batches += 1

        if idx % 100 == 0 and idx > 0:
            print(f'Epoch {epoch}, Batch {idx}: Loss: {loss.item():.4f}')

    print(f'Epoch {epoch}: Train Loss: {total_loss/total_batches:.4f}')

def evaluate(dataloader, model, criterion):
    model.eval()
    total_loss = 0
    total_batches = 0

    model_device = next(model.parameters()).device

    with torch.no_grad():
        for idx, (data, targets, src_padding_mask) in enumerate(dataloader):
            data, targets, src_padding_mask = data.to(model_device), targets.to(model_device), src_padding_mask.to(model_device)
            src_mask = generate_square_subsequent_mask(data.size(1)).to(model_device)

            predicted_logits = model(data, src_mask, src_padding_mask)

            # --- CHANGE THIS LINE ---
            loss = criterion(predicted_logits.reshape(-1, VOCAB_SIZE), targets.reshape(-1))

            total_loss += loss.item()
            total_batches += 1

            del data, targets, predicted_logits, src_mask, src_padding_mask
            torch.cuda.empty_cache()

    avg_loss = total_loss / total_batches
    perplexity = math.exp(avg_loss)
    return avg_loss, perplexity

In [ ]:
!rm -rf /root/.cache/torch/text/datasets/IMDB

In [ ]:
# Train the model
for epoch in range(1, num_epochs + 1):
    train(train_dataloader, model, criterion, optimizer, epoch)
    val_loss = evaluate(test_dataloader, model, criterion)
    print(f'Epoch {epoch}: Test Loss: {val_loss:.4f}')

/usr/local/lib/python3.11/dist-packages/torch/nn/functional.py:5109: UserWarning: Support for mismatched key_padding_mask and attn_mask is deprecated. Use same type for both instead.
  warnings.warn(


Epoch 1, Batch 100: Loss: 9.5896
Epoch 1, Batch 200: Loss: 7.6290
Epoch 1, Batch 300: Loss: 7.0269
Epoch 1: Train Loss: 8.3399
Epoch 1: Test Loss: 6.6089
Epoch 2, Batch 100: Loss: 6.5115
Epoch 2, Batch 200: Loss: 6.1716
Epoch 2, Batch 300: Loss: 6.1308
Epoch 2: Train Loss: 6.3033
Epoch 2: Test Loss: 6.0714


In [ ]:
# --- 5. Quantization (Post-Training Dynamic) ---
print("\nApplying quantization...")

# Create a copy of the model for quantization
# Use the same parameters to create the model_to_quantize
model_to_quantize = TransformerTextGenerator(VOCAB_SIZE, EMBED_DIM, NHEAD, NUM_LAYERS, DIM_FEEDFORWARD, DROPOUT)
model_to_quantize.load_state_dict(model.state_dict())
model_to_quantize.eval() # Set to eval mode for quantization

# Apply dynamic quantization to Linear layers and Transformer specific modules
quantized_model = torch.quantization.quantize_dynamic(
    model_to_quantize, {nn.Linear}, dtype=torch.qint8 # <--- ONLY QUANTIZE NN.LINEAR
)
print("Quantization complete.")


Applying quantization...
Quantization complete.


In [ ]:
print("\n--- Original Model Statistics ---")
# Print model architecture
print(model)
# Count parameters
original_total_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"Total trainable parameters: {original_total_params:,}")
# Calculate in-memory size
original_size_mb = sum(p.element_size() * p.numel() for p in model.parameters()) / (1024**2)
print(f"In-memory size: {original_size_mb:.2f} MB")


print("\n--- Quantized Model Statistics ---")
# Print quantized model architecture (note the 'Quantized' prefix on some layers)
print(quantized_model)
# Count parameters (note: quantized parameters might report different sizes or types)
# For accurate comparison, it's better to look at memory footprint after quantization.
quantized_total_params = sum(p.numel() for p in quantized_model.parameters() if p.requires_grad)
print(f"Total trainable parameters (quantized): {quantized_total_params:,}")
# Calculate in-memory size (will reflect reduced precision for quantized layers)
quantized_size_mb = sum(p.element_size() * p.numel() for p in quantized_model.parameters()) / (1024**2)
print(f"In-memory size: {quantized_size_mb:.2f} MB")

print(f"\nMemory Reduction: {((original_size_mb - quantized_size_mb) / original_size_mb) * 100:.2f}%")


--- Original Model Statistics ---
TransformerTextGenerator(
  (embedding): Embedding(100686, 64, padding_idx=1)
  (pos_encoder): PositionalEncoding(
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (transformer_decoder): TransformerDecoder(
    (layers): ModuleList(
      (0-2): 3 x TransformerDecoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=64, out_features=64, bias=True)
        )
        (multihead_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=64, out_features=64, bias=True)
        )
        (linear1): Linear(in_features=64, out_features=128, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
        (linear2): Linear(in_features=128, out_features=64, bias=True)
        (norm1): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
        (norm2): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
        (norm3): LayerNorm((64,), eps=1e-05, elementwi

In [ ]:
# --- 6. Text Generation Example (Adapted for Transformer) ---
print("\nExample text generation:")

def generate_text(model, vocab, start_text, max_len=50, temperature=0.8):
    model.eval()
    input_ids = [BOS_IDX] + text_pipeline(start_text)
    generated_ids = list(input_ids)

    model_device = next(model.parameters()).device

    with torch.no_grad():
        for _ in range(max_len):
            current_sequence_tensor = torch.tensor([generated_ids], dtype=torch.long).to(model_device)

            src_mask = generate_square_subsequent_mask(current_sequence_tensor.size(1)).to(model_device)

            # --- ADD THIS LINE ---
            src_padding_mask = (current_sequence_tensor == PAD_IDX).to(model_device)
            # This mask is usually all False for single sequence generation (no padding within it)

            output_logits = model(current_sequence_tensor, src_mask, src_padding_mask) # <--- Pass src_padding_mask here

            prediction_logits = output_logits[:, -1, :] / temperature
            probabilities = F.softmax(prediction_logits, dim=-1)

            next_token_id = torch.multinomial(probabilities, num_samples=1).item()

            generated_ids.append(next_token_id)
            if next_token_id == EOS_IDX:
                break

    generated_text = ' '.join(vocab.lookup_tokens(generated_ids))
    generated_text = generated_text.replace(vocab.lookup_token(BOS_IDX), '')
    generated_text = generated_text.replace(vocab.lookup_token(EOS_IDX), '')
    generated_text = generated_text.replace(vocab.lookup_token(PAD_IDX), '')
    return ' '.join(generated_text.split())


Example text generation:


In [ ]:
# Test generation with the original (full precision) model
start_phrase = "This movie was"
generated_sentence = generate_text(model, vocab, start_phrase, max_len=30)
print(f"Prompt: '{start_phrase}'\nGenerated (Original): '{generated_sentence}'")

Prompt: 'This movie was'
Generated (Original): 'this movie was that is not so this movie would be , and in the ' s girlfriend . i script is not if she are not bad . i was [1953/7] ('


In [ ]:
# Test generation with the quantized model
start_phrase_quant = "I did not like"
generated_sentence_quant = generate_text(quantized_model, vocab, start_phrase_quant, max_len=30)
print(f"Prompt: '{start_phrase_quant}'\nGenerated (Quantized): '{generated_sentence_quant}'")

Prompt: 'I did not like'
Generated (Quantized): 'i did not like its the uckridge . i and i have ever appear , but the case , i have the young film ? it is not . it look the young e'


In [ ]:
# --- After 'Evaluating quantized model...' print statement ---

print("\n--- Model Comparison ---")

# A. Evaluate Original Model
print("Evaluating original model...")
original_loss, original_perplexity = evaluate(test_dataloader, model, criterion)
print(f"Original Model Test Loss: {original_loss:.4f}")
print(f"Original Model Test Perplexity: {original_perplexity:.2f}")


--- Model Comparison ---
Evaluating original model...
Original Model Test Loss: 6.0714
Original Model Test Perplexity: 433.30


In [ ]:
TEST_BATCH_SIZE = 2

test_dataloader = DataLoader(test_iter, batch_size=TEST_BATCH_SIZE, shuffle=False, collate_fn=collate_batch)

In [ ]:
# B. Evaluate Quantized Model
print("\nEvaluating quantized model...")
quantized_loss, quantized_perplexity = evaluate(test_dataloader, quantized_model, criterion)
print(f"Quantized Model Test Loss: {quantized_loss:.4f}")
print(f"Quantized Model Test Perplexity: {quantized_perplexity:.2f}")


Evaluating quantized model...


In [ ]:
# C. Compare Model Sizes (in-memory)
original_size_mb = sum(p.element_size() * p.numel() for p in model.parameters()) / (1024**2)
quantized_size_mb = sum(p.element_size() * p.numel() for p in quantized_model.parameters()) / (1024**2)

print(f"\nOriginal Model Size (in-memory): {original_size_mb:.2f} MB")
print(f"Quantized Model Size (in-memory): {quantized_size_mb:.2f} MB")
print(f"Memory Reduction: {((original_size_mb - quantized_size_mb) / original_size_mb) * 100:.2f}%")

In [ ]:
# D. Compare Model Sizes (disk - optional, requires saving)
# This requires saving the models to disk first:
# torch.save(model.state_dict(), "original_model.pth")
# torch.save(quantized_model.state_dict(), "quantized_model.pth")
# original_disk_size_mb = os.path.getsize("original_model.pth") / (1024**2)
# quantized_disk_size_mb = os.path.getsize("quantized_model.pth") / (1024**2)
# print(f"Original Model Size (on disk): {original_disk_size_mb:.2f} MB")
# print(f"Quantized Model Size (on disk): {quantized_disk_size_mb:.2f} MB")